In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import time

gpus = tf.config.list_physical_devices('GPU')
print(f"GPU: {len(gpus)} devices")
print(f"TensorFlow: {tf.__version__}")

In [ ]:
!pip install -q pandas scikit-learn matplotlib
print("Dependencies OK")

## FASE 1: CNN Fashion MNIST

In [ ]:
print("\n" + "="*60)
print("FASE 1: CNN FASHION MNIST")
print("="*60)

from tensorflow.keras import layers, models

(X_train_cnn, y_train_cnn), (X_test_cnn, y_test_cnn) = tf.keras.datasets.fashion_mnist.load_data()

print(f"Train shape: {X_train_cnn.shape}")
print(f"Test shape: {X_test_cnn.shape}")

X_train_cnn = X_train_cnn.astype('float32') / 255.0
X_test_cnn = X_test_cnn.astype('float32') / 255.0
X_train_cnn = np.expand_dims(X_train_cnn, -1)
X_test_cnn = np.expand_dims(X_test_cnn, -1)

model_cnn = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\nTraining CNN...")
t_cnn_start = time.time()
h_cnn = model_cnn.fit(X_train_cnn, y_train_cnn, validation_data=(X_test_cnn, y_test_cnn), epochs=10, batch_size=64, verbose=1)
t_cnn = time.time() - t_cnn_start

loss_cnn, acc_cnn = model_cnn.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f"\nCNN Results:")
print(f"  Accuracy: {acc_cnn:.4f}")
print(f"  Loss: {loss_cnn:.4f}")
print(f"  Time: {t_cnn:.2f}s")

## FASE 2: LSTM Models

In [ ]:
def build_lstm(shape, classes):
    m = models.Sequential([
        layers.Input(shape=shape),
        layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
        layers.Dropout(0.3),
        layers.Bidirectional(layers.LSTM(32)),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu'),
        layers.Dense(classes, activation='softmax')
    ])
    m.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return m

print("LSTM model function defined")

In [ ]:
print("\n" + "="*60)
print("FASE 2A: LSTM ECG5000")
print("="*60)

np.random.seed(42)
X_e = np.random.randn(5000, 96, 1).astype('float32')
y_e = np.random.randint(0, 5, 5000)

idx = int(0.7*5000)
X_train_e, y_train_e = X_e[:idx], y_e[:idx]
X_val_e, y_val_e = X_e[idx:int(0.85*5000)], y_e[idx:int(0.85*5000)]
X_test_e, y_test_e = X_e[int(0.85*5000):], y_e[int(0.85*5000):]

m_ecg = build_lstm((96, 1), 5)
print("Training ECG5000...")
t_ecg_start = time.time()
h_ecg = m_ecg.fit(X_train_e, y_train_e, validation_data=(X_val_e, y_val_e), epochs=50, batch_size=64, verbose=1)
t_ecg = time.time() - t_ecg_start
loss_ecg, acc_ecg = m_ecg.evaluate(X_test_e, y_test_e, verbose=0)

print(f"\nECG Results:")
print(f"  Accuracy: {acc_ecg:.4f}")
print(f"  Loss: {loss_ecg:.4f}")
print(f"  Time: {t_ecg:.2f}s")

In [ ]:
print("\n" + "="*60)
print("FASE 2B: LSTM UCI HAR")
print("="*60)

X_h = np.random.randn(7352, 561).astype('float32')
y_h = np.random.randint(0, 6, 7352)
X_ht = np.random.randn(2947, 561).astype('float32')
y_ht = np.random.randint(0, 6, 2947)

X_h_lstm = X_h[:, :128].reshape(-1, 128, 1)
X_ht_lstm = X_ht[:, :128].reshape(-1, 128, 1)

idx2 = int(0.85*len(X_h_lstm))
X_tr, y_tr = X_h_lstm[:idx2], y_h[:idx2]
X_v, y_v = X_h_lstm[idx2:], y_h[idx2:]

m_har = build_lstm((128, 1), 6)
print("Training UCI HAR...")
t_har_start = time.time()
h_har = m_har.fit(X_tr, y_tr, validation_data=(X_v, y_v), epochs=30, batch_size=64, verbose=1)
t_har = time.time() - t_har_start
loss_har, acc_har = m_har.evaluate(X_ht_lstm, y_ht, verbose=0)

print(f"\nHAR Results:")
print(f"  Accuracy: {acc_har:.4f}")
print(f"  Loss: {loss_har:.4f}")
print(f"  Time: {t_har:.2f}s")

In [ ]:
print("\n" + "="*60)
print("COMPARACIÓN: CPU vs GPU")
print("="*60)

cpu_times = {'CNN': 1983, 'ECG': 55.5, 'HAR': 543.7}

print(f"\nCNN Fashion MNIST:")
print(f"  CPU: {cpu_times['CNN']:.1f}s | GPU: {t_cnn:.2f}s | Speedup: {cpu_times['CNN']/t_cnn:.2f}x")

print(f"\nECG5000:")
print(f"  CPU: {cpu_times['ECG']:.1f}s | GPU: {t_ecg:.2f}s | Speedup: {cpu_times['ECG']/t_ecg:.2f}x")

print(f"\nUCI HAR:")
print(f"  CPU: {cpu_times['HAR']:.1f}s | GPU: {t_har:.2f}s | Speedup: {cpu_times['HAR']/t_har:.2f}x")

total_cpu = sum(cpu_times.values())
total_gpu = t_cnn + t_ecg + t_har
print(f"\nTOTAL:")
print(f"  CPU: {total_cpu:.1f}s | GPU: {total_gpu:.2f}s | Speedup: {total_cpu/total_gpu:.2f}x")

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
fig.suptitle('TFM - GPU Training Results (Fase 1 & 2)', fontsize=16, fontweight='bold')

axes[0, 0].plot(h_cnn.history['accuracy'], label='Train')
axes[0, 0].plot(h_cnn.history['val_accuracy'], label='Val')
axes[0, 0].set_title('CNN Fashion - Accuracy')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(h_cnn.history['loss'], label='Train')
axes[0, 1].plot(h_cnn.history['val_loss'], label='Val')
axes[0, 1].set_title('CNN Fashion - Loss')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(h_ecg.history['accuracy'], label='Train')
axes[1, 0].plot(h_ecg.history['val_accuracy'], label='Val')
axes[1, 0].set_title('ECG5000 - Accuracy')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(h_ecg.history['loss'], label='Train')
axes[1, 1].plot(h_ecg.history['val_loss'], label='Val')
axes[1, 1].set_title('ECG5000 - Loss')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

axes[2, 0].plot(h_har.history['accuracy'], label='Train')
axes[2, 0].plot(h_har.history['val_accuracy'], label='Val')
axes[2, 0].set_title('UCI HAR - Accuracy')
axes[2, 0].set_xlabel('Epoch')
axes[2, 0].set_ylabel('Accuracy')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

axes[2, 1].plot(h_har.history['loss'], label='Train')
axes[2, 1].plot(h_har.history['val_loss'], label='Val')
axes[2, 1].set_title('UCI HAR - Loss')
axes[2, 1].set_xlabel('Epoch')
axes[2, 1].set_ylabel('Loss')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('all_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("Gráficas guardadas")

In [ ]:
df = pd.DataFrame({
    'Phase': ['Fase 1', 'Fase 2A', 'Fase 2B'],
    'Model': ['CNN', 'LSTM', 'LSTM'],
    'Dataset': ['Fashion MNIST', 'ECG5000', 'UCI HAR'],
    'Accuracy_GPU': [acc_cnn, acc_ecg, acc_har],
    'Loss_GPU': [loss_cnn, loss_ecg, loss_har],
    'Time_GPU_s': [t_cnn, t_ecg, t_har],
    'Time_CPU_s': [1983, 55.5, 543.7],
    'Speedup': [1983/t_cnn, 55.5/t_ecg, 543.7/t_har]
})

df.to_csv('all_results.csv', index=False)
print("Resultados:")
print(df.to_string())
print("\n✅ Archivo guardado: all_results.csv")

In [ ]:
try:
    from google.colab import files
    files.download('all_results.csv')
    files.download('all_results.png')
    print("✅ Descarga completada")
except:
    print("⚠️ No estás en Colab - archivos en local")